In [3]:
import pandas as pd
import glob
from pathlib import Path
import numpy as np
from numpy.random import default_rng
rng = default_rng()


folder = "../../results_all/app_history_vm/"
filePrefix = "../../results_all/app_bootstrap_vm/app_bootstrap_vm"
print(folder)

start = 0.05
end = 0.2

numberOfSamples = 10000
CIsmall = 1

../../results_all/app_history_vm/


In [4]:
def resample(perfRuntimes1: pd.DataFrame,
            perfRuntimes2: pd.DataFrame,
            instanceRuns: np.ndarray,
            samples: int,
            numberOfSamples: int) -> np.ndarray:
    """
        Resamples performances using hierarchical bootstrapping for building confindence intervals

        Builds a tensor of random indices of a form numberOfSamples * instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Then uses these indices to choose from a performance runtimes tensor with a form instanceRunsNumber * suiteRunsNumber * numberOfIterations.
        Finally, reshapes resulting tensor to a matrix of form numberOfSamples * ( instanceRunsNumber * suiteRunsNumber * numberOfIterations)
        and calculates performance differences between medians.

        Parameters
        ----------
        perfRuntimes1 : performance runtimes of the first version.
        perfRuntimes2 : performance runtimes of the second version.
        instanceRuns : array of instanceRun numbers
        samples : number of drawn alues

        Returns
        -------
        np.ndarray
            array of performance differences with a shape numberOfSample * 1.

        """
    instanceRunsNumber = instanceRuns.shape[0]

    allRuntimes1 = np.ndarray((instanceRunsNumber, samples))
    allRuntimes2 = np.ndarray((instanceRunsNumber, samples))


    for instanceRun in instanceRuns:
        allRuntimes1[instanceRun - 1]= perfRuntimes1.loc[(perfRuntimes1['run'] == instanceRun),
                                                'latency (ms)'].to_numpy()
        allRuntimes2[instanceRun - 1]= perfRuntimes2.loc[(perfRuntimes2['run'] == instanceRun),
                                                'latency (ms)'].to_numpy()

    medians = []
    for i in range(numberOfSamples):
        #if (i % 1000 == 0):
        #    print(f"i is {i}")
        #Generate Random Arrays
        currentInstanceRun = rng.choice(instanceRuns, size=(instanceRunsNumber)) - 1
        currentRuntimes1 = rng.integers(samples, size=(samples, instanceRunsNumber))
        currentRuntimes2 = rng.integers(samples, size=(samples, instanceRunsNumber))
        #Bulk selection
        tmp1 = allRuntimes1[currentInstanceRun, currentRuntimes1]
        tmp1 = tmp1.reshape((instanceRunsNumber * samples))

        tmp2 = allRuntimes2[currentInstanceRun, currentRuntimes2]
        tmp2 = tmp2.reshape((instanceRunsNumber * samples))

        # Get median for both lists
        med1 = np.median(tmp1, axis=0)
        med2 = np.median(tmp2, axis=0)
        medians.append(med2/med1)

    return medians

def bootstrap(perfRuntimes1: pd.DataFrame,
            perfRuntimes2: pd.DataFrame):

    instanceRuns = perfRuntimes1.run.unique()
    numberOfInstanceRuns = len(instanceRuns)
    print(f"    Found {numberOfInstanceRuns} runs")
    R = resample(perfRuntimes1=perfRuntimes1,
                 perfRuntimes2=perfRuntimes2,
                 instanceRuns=instanceRuns,
                 samples=int(len(perfRuntimes1) / numberOfInstanceRuns),
                 numberOfSamples=numberOfSamples)

    print(f"    Bootstrapping done ({len(R)} elements in R).")

    # Find conf. intervals
    R.sort()

    small = int((numberOfSamples * CIsmall) / 100 / 2)
    if small == 0:
        small  = 1

    minSmall = R[small-1]
    minSmall = (minSmall - 1) * 100
    maxSmall = R[numberOfSamples-small-1]
    maxSmall = (maxSmall - 1) * 100
    return minSmall, maxSmall


all_runs = glob.glob(folder + "*")

for fullRunFolder in all_runs:
    runFolder = fullRunFolder + "/"
    all_numbers = glob.glob(runFolder + "*")
    for fullNumberFolder in all_numbers:

        all_files = glob.glob(fullNumberFolder + "/*")
        size = len(all_files)

        if size != 22:
            print(f"{size} elements in folder {fullNumberFolder}, skip.")
        else:
            tmp = fullNumberFolder.split("\\")

            #Parse run and commit number from path structure
            run = tmp[len(tmp)-2]
            run=run[3:]
            commitNumber = tmp[len(tmp)-1]
            print(f"Check Run {run} Number {commitNumber}")

            filename = Path(filePrefix + "_" + commitNumber + ".csv")
            skip = False

            if filename.is_file():
                print("  Already aggregated, skip")
            else:
                print("  Aggregating...")
                rows = []
                for i in range(3):
                    currFolder = folder + "run" + str((i+1)) + "/" + str(commitNumber)
                    tmp = glob.glob(currFolder + "/*")
                    noOfFiles = len(tmp)
                    print(f"    Current folder: {currFolder} with {noOfFiles} files")

                    if noOfFiles != 22:
                        print(f"ERROR: {noOfFiles} elements in folder {currFolder}, skip.")
                        skip = True
                    else:

                        #Read latency file for both types (old and new)
                        inserts1Old = pd.read_csv(currFolder + "/latenciesInserts1Old.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["batch","latency (ms)"])
                        inserts1Old.sort_values(by=["batch"], inplace=True)
                        number = len(inserts1Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        inserts1Old = inserts1Old.iloc[removeFirst:-removeLast , :]

                        for index, row in inserts1Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        inserts1New = pd.read_csv(currFolder + "/latenciesInserts1New.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["batch","latency (ms)"])
                        inserts1New.sort_values(by=["batch"], inplace=True)
                        number = len(inserts1New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        inserts1New = inserts1New.iloc[removeFirst:-removeLast , :]

                        for index, row in inserts1New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        #Read latency file for both types (old and new)
                        inserts2Old = pd.read_csv(currFolder + "/latenciesInserts2Old.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["batch","latency (ms)"])
                        inserts2Old.sort_values(by=["batch"], inplace=True)
                        number = len(inserts2Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        inserts2Old = inserts2Old.iloc[removeFirst:-removeLast , :]

                        for index, row in inserts2Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        inserts2New = pd.read_csv(currFolder + "/latenciesInserts2New.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["batch","latency (ms)"])
                        inserts2New.sort_values(by=["batch"], inplace=True)
                        number = len(inserts2New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        inserts2New = inserts2New.iloc[removeFirst:-removeLast , :]

                        for index, row in inserts2New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["batch"],
                                "type" : "inserts",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries1Old = pd.read_csv(currFolder + "/latenciesQueries1Old.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["query","latency (ms)"])
                        queries1Old.sort_values(by=["query"], inplace=True)
                        number = len(queries1Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries1Old = queries1Old.iloc[removeFirst:-removeLast , :]


                        for index, row in queries1Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "simple queries",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries2Old = pd.read_csv(currFolder + "/latenciesQueries2Old.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["query","latency (ms)"])
                        queries2Old.sort_values(by=["query"], inplace=True)

                        number = len(queries2Old.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries2Old = queries2Old.iloc[removeFirst:-removeLast , :]

                        for index, row in queries2Old.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "group-by queries",
                                "version" : "base",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries1New = pd.read_csv(currFolder + "/latenciesQueries1New.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["query","latency (ms)"])
                        queries1New.sort_values(by=["query"], inplace=True)

                        number = len(queries1New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries1New = queries1New.iloc[removeFirst:-removeLast , :]

                        for index, row in queries1New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "simple queries",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })

                        queries2New = pd.read_csv(currFolder + "/latenciesQueries2New.csv",
                                         header=None,
                                         delim_whitespace=True,
                                         names=["query","latency (ms)"])
                        queries2New.sort_values(by=["query"], inplace=True)

                        number = len(queries2New.index)
                        removeFirst = int(start * number)
                        removeLast = int(end * number)
                        queries2New = queries2New.iloc[removeFirst:-removeLast , :]

                        for index, row in queries2New.iterrows():
                            rows.append({
                                "run":i+1,
                                "number": row["query"],
                                "type" : "group-by queries",
                                "version" : "variation",
                                "latency (ms)" : row["latency (ms)"]
                            })
                if not skip:

                    #Run bootstrapping for each type
                    result = []
                    df_latencies = pd.DataFrame(rows)
                    for type in df_latencies.type.unique():
                        print(f"  Running bootstrapping for type {type}...")

                        benchmarkMeasurements = df_latencies.loc[(df_latencies['type'].str.startswith(type, na=False))]
                        perfRuntimes1 = benchmarkMeasurements.loc[(benchmarkMeasurements['version'] == "base")]
                        perfRuntimes2 = benchmarkMeasurements.loc[(benchmarkMeasurements['version'] == "variation")]

                        print(f"  Find median perf change for type {type}...")
                        median1 = perfRuntimes1["latency (ms)"].median()
                        median2 = perfRuntimes2["latency (ms)"].median()
                        diff = ((median2 / median1) - 1) * 100

                        print(f"  Running bootstrapping for type {type}...")
                        instanceRuns = benchmarkMeasurements.run.unique()
                        numberOfInstanceRuns = len(instanceRuns)
                        min, max = bootstrap(perfRuntimes1, perfRuntimes2)
                        print(f"    Min: {min} Max: {max} (Width: {max-min})")
                        result.append({
                            "commit":commitNumber,
                            "type": type,
                            "min" : min,
                            "med" : diff,
                            "max" : max
                        })

                    # Convert to data frame
                    df_result = pd.DataFrame(result)
                    # Save to file
                    df_result.to_csv(str(filename))

Check Run 1 Number 1
  Aggregating...
    Current folder: ../../results_all/app_history_vm/run1/1 with 22 files
    Current folder: ../../results_all/app_history_vm/run2/1 with 22 files
    Current folder: ../../results_all/app_history_vm/run3/1 with 22 files
  Running bootstrapping for type inserts...
  Find median perf change for type inserts...
  Running bootstrapping for type inserts...
    Found 3 runs
    Bootstrapping done (10000 elements in R).
    Min: -0.5677039226128167 Max: 1.9646954813379747 (Width: 2.5323994039507913)
  Running bootstrapping for type simple queries...
  Find median perf change for type simple queries...
  Running bootstrapping for type simple queries...
    Found 3 runs
    Bootstrapping done (10000 elements in R).
    Min: -0.08567302504857155 Max: 1.694792456635219 (Width: 1.7804654816837906)
  Running bootstrapping for type group-by queries...
  Find median perf change for type group-by queries...
  Running bootstrapping for type group-by queries...
  